# EdS extrapolated gaussian analysis 
Creating extrapolated gaussian profiles and plotting them alongside fitted curves


In [ ]:
# Import packages
import numpy as np
import matplotlib.pyplot as plt
import scipy 
import scipy.stats
import scipy.optimize as scopt
from scipy.stats import norm
from scipy.stats import skew
from scipy.stats import skewnorm
from scipy.optimize import curve_fit as scurv


## Define gaussian curve for fitting

In [ ]:
def gaussian(x, mean, amplitude, standard_deviation):
    return amplitude * np.exp( - (x - mean)**2 / (2*standard_deviation ** 2))

## Load data for use in plotting

In [ ]:
def data_load(rad):
    z = []
    deltas = []

    q = open(f'Text_data/z_{rad}.txt', 'r')
    lines = q.readlines()
    for i in lines: 
        z.append(i.split(  )[0])
    q.close()
    z_arr = np.array(z).astype(float)
    sort = np.argsort(z_arr)
    z_arr = z_arr[sort]

    k = open(f'Text_Data/delta_{rad}.txt', 'r')
    lines = k.readlines()
    for i in lines:
        deltas.append(i.split(  ))

    k.close()
    delta_arr = np.array(deltas).astype(float)
    delta_arr = delta_arr[sort]
    
    var_arr = np.ones(len(z_arr))

    skew_arr = np.ones(len(z_arr))
    z = z_arr[13]
    delta = delta_arr[13]
    
    bin_heights, bin_borders = np.histogram(delta_arr[13], bins = 500, range = (-1,1))
    bin_centers = bin_borders[:-1] + np.diff(bin_borders) / 2
    
    popt, pcov = scurv(gaussian, bin_centers, bin_heights, p0 = [0, 1 ,0.1], maxfev = 30000)
    a = 1/(1+z)
    b = popt[2]/a
    
    perr = np.sqrt(np.diag(pcov))
    b_err = perr[2]/a
    
    return z_arr, delta_arr, b, b_err 

## Create histogram of loaded data, fit curve and add extrapolated, return array of data

In [ ]:
def extrapolation(z_arr, delta_arr, b, rad):
    
    analytic = np.array([])
    modified = np.array([])
    
    x = np.linspace(-1, 1,10000)
    
    for i in range(13):
        
        
        z = z_arr[i] 
        a = 1/(1+z)
    

        # Find mean and standard deviation of delta array 
        mean, std = norm.fit(delta_arr[i])


        # Plot histogram of overdensity data and save
        z = np.round(z, decimals = 2)
        n_bin = 70
        plt.clf()
        hist =plt.hist(delta_arr[i], bins = n_bin, range = (-1,1), label = 'Overdensities', color = 'g')
        plt.title(f'Overdensity Histogram for z = {z}')
        plt.xlabel('Overdensity $\delta$')
        plt.ylabel('Frequency')
           
        
        bin_heights, bin_borders = np.histogram(delta_arr[i], bins = n_bin, range = (-1,1))
        bin_centers = bin_borders[:-1] + np.diff(bin_borders) / 2 
        
        popt, pcov = scurv(gaussian, bin_centers, bin_heights, p0 = [0, 1 ,0.1], maxfev = 30000)
        
        modified = np.append(modified, popt[2])
        
        plt.plot(x, gaussian(x, *popt), label = 'Fitted', color = 'blue')
        
        popt, pcov = scurv(gaussian, bin_centers, bin_heights, p0 = [0, 1 ,0.1], maxfev = 30000)
        
        analytic = np.append(analytic, b*a)
        
        plt.plot(x, gaussian(x, popt[0], popt[1], b*a), label = 'Linear', color = 'r')
        
        plt.xlabel('Overdensity $\delta$')
        plt.ylabel('Frequency')
        plt.title(f'Linear Extrapolation of Overdensity Values for z = {z}')
        plt.legend()
        plt.savefig(f'Extrapolated/Extrapolated_{rad}Mpc_redshift_{z}.jpeg', dpi =200)
        plt.show()
        
    return modified, analytic 
    
    

## Subplots to use in presentation and report for chosen z vals

In [ ]:
def subplots_extrapolated(z_arr, delta_arr, b, rad):
    
    ran = (-1,1)
    x = np.linspace(-1, 1, 1000)
    
    fig, ((ax1,ax2),(ax3,ax4)) = plt.subplots(figsize = (8,4), nrows=2, ncols=2, sharex = True)
    
    a = 1/(1+z_arr)
    z = np.round(z_arr, decimals = 2)
    
    bin_heights, bin_borders = np.histogram(delta_arr[0], bins = 30 , range = ran)
    bin_centers = bin_borders[:-1] + np.diff(bin_borders) / 2 
    hist = ax4.hist(delta_arr[0], bins = 30, range = ran, color = 'g' )
    popt, pcov = scurv(gaussian, bin_centers, bin_heights, p0 = [0, 1 ,0.1], maxfev = 30000)
    ax4.plot(x, gaussian(x, *popt), color = 'r')
    popt, pcov = scurv(gaussian, bin_centers, bin_heights, p0 = [0, 1 ,0.1], maxfev = 30000)
    ax4.plot(x, gaussian(x, popt[0], popt[1], b*a[0]), color = 'b')
    ax4.text(0.7,0.9, 'z = 0.0', transform=ax4.transAxes)
    
    
    bin_heights, bin_borders = np.histogram(delta_arr[3], bins = 50 , range = ran)
    bin_centers = bin_borders[:-1] + np.diff(bin_borders) / 2 
    hist = ax3.hist(delta_arr[3], bins = 50, range = ran, color = 'g' )
    popt, pcov = scurv(gaussian, bin_centers, bin_heights, p0 = [0, 1 ,0.1], maxfev = 30000)
    ax3.plot(x, gaussian(x, *popt), color = 'r')
    popt, pcov = scurv(gaussian, bin_centers, bin_heights, p0 = [0, 1 ,0.1], maxfev = 30000)
    ax3.plot(x, gaussian(x, popt[0], popt[1], b*a[3]), color = 'b')
    ax3.text(0.7,0.9, 'z = 0.38', transform=ax3.transAxes)
    
    bin_heights, bin_borders = np.histogram(delta_arr[7], bins = 80 , range = ran)
    bin_centers = bin_borders[:-1] + np.diff(bin_borders) / 2 
    hist = ax2.hist(delta_arr[7], bins = 80, range = ran, color = 'g' )
    popt, pcov = scurv(gaussian, bin_centers, bin_heights, p0 = [0, 1 ,0.1], maxfev = 30000)
    ax2.plot(x, gaussian(x, *popt), color = 'r')
    popt, pcov = scurv(gaussian, bin_centers, bin_heights, p0 = [0, 1 ,0.1], maxfev = 30000)
    ax2.plot(x, gaussian(x, popt[0], popt[1], b*a[7]), color = 'b')
    ax2.text(0.7,0.9, 'z = 1.55 ', transform=ax2.transAxes)
    
    bin_heights, bin_borders = np.histogram(delta_arr[13], bins = 100 , range = ran)
    bin_centers = bin_borders[:-1] + np.diff(bin_borders) / 2 
    hist = ax1.hist(delta_arr[13], bins = 100, range = ran, color = 'g' )
    popt, pcov = scurv(gaussian, bin_centers, bin_heights, p0 = [0, 1 ,0.1], maxfev = 30000)
    ax1.plot(x, gaussian(x, *popt), color = 'r', label = 'Fitted')
    popt, pcov = scurv(gaussian, bin_centers, bin_heights, p0 = [0, 1 ,0.1], maxfev = 30000)
    ax1.plot(x, gaussian(x, popt[0], popt[1], b*a[13]), color = 'b', label = 'Extrapolated')
    ax1.text(0.7,0.9, 'z = 10.76', transform=ax1.transAxes)
    
    fig.text(0.5, 0.02, 'Overdensity $\delta$', ha='center', fontsize=14)
    fig.text(0.04, 0.5, 'Frequency', va='center', rotation='vertical', fontsize=14)
    fig.subplots_adjust(wspace=0.15, hspace=0.15)
    fig.legend(bbox_to_anchor = (0.595, 0))
    print((f'Analytic Prediction for z = {z[0]}, {z[3]}, {z[7]}, {z[13]}'))
    plt.savefig('Subplots\Gaussian_Extrapolated_Subplots', dpi =200)
    plt.show()


## Write out data from plotting function 

In [ ]:
def write_data(modified, analytic, rad):
    
    outfile = open(f'Text_Data/Extrapolation Data/modified_{rad}Mpc.txt', 'w')

    for i in range(len(modified)):
        outfile.write(str(modified[i]) + "\n")

    outfile.close()

    outfile = open(f'Text_Data/Extrapolation Data/analytic_{rad}Mpc.txt', 'w')

    for i in range(len(analytic)):
        outfile.write(str(analytic[i]) + "\n")

    outfile.close()
    

In [ ]:
z_arr, delta_arr, b, b_err = data_load(30)
modified, analytic = extrapolation(z_arr, delta_arr, b, 30)
subplots_extrapolated(z_arr, delta_arr, b, 30)
